In [1]:
import braindecode

In [2]:
import mne
from scipy.io import loadmat
import scipy
import sklearn
import numpy as np
import pandas as pd
import glob
from mne.decoding import CSP
import os

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, LeaveOneGroupOut, StratifiedShuffleSplit
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as lda

In [4]:
import warnings
warnings.filterwarnings('ignore') # to ignore warnings

In [5]:
verbose = False                    # global variable to suppress output display of MNE functions
mne.set_log_level(verbose=verbose) # to suppress large info outputs

In [6]:
n_jobs = None  # for multicore parallel processing, set it to 1 if cause memory issues, for full utilization set to -1

## Data Loading and Conversion to MNE Datatypes
[Mike Cohen Tutorials link for EEG Preprocessing](https://www.youtube.com/watch?v=uWB5tjhataY&list=PLn0OLiymPak2gDD-VDA90w9_iGDgOOb2o)

In [7]:
current_folder = globals()['_dh'][0]  # a hack to get path of current folder in which jupyter file is located
data_path = os.path.join(current_folder, r'C:\Users\User\Documents\GitHub\Frequency-Adaptive-Temporal-Kernel-EEGNet\Data')

In [8]:
training_files   = glob.glob(data_path + '/*T.mat')
len(training_files)     # if  return zero,then no file is loaded

10

## Lets Append Epochs

In [9]:
def get_mne_epochs_complete(files_paths, verbose=verbose, t_start=2, fs=512, mode='train'):
    '''
    similar to get_mne_epochs, just appends data from all relevant files together to give a single
    epoch object
    '''
    eeg_data = []
    for filepath in files_paths:
        mat_data = loadmat(filepath)
        eeg_data.extend(mat_data['RawEEGData'])

    idx_start = fs*t_start      # fs*ts
    eeg_data = np.array(eeg_data)
    eeg_data = eeg_data[:, :, idx_start:]
    event_id = {'left-hand': 0, 'right-hand': 1} # pytorch expects labels in [0, n_classes-1]
    channel_names = ['F3', 'FC3', 'C3', 'CP3', 'P3', 'FCz', 'CPz', 'F4', 'FC4', 'C4', 'CP4', 'P4']
    info = mne.create_info(ch_names=channel_names, sfreq=fs, ch_types='eeg')
    epochs = mne.EpochsArray(eeg_data, info, verbose=verbose, tmin=t_start-3.0)
    epochs.set_montage('standard_1020')
    epochs.filter(1., None) # required be ICA, (7-30 Hz) later
    epochs.apply_baseline(baseline=(-.250, 0)) # linear baseline correction
    
    if mode == 'train': # this in only applicable for training data
        labels = []
        for filepath in files_paths:
            mat_data = loadmat(filepath)
            labels.extend(mat_data['Labels'].ravel() - 1)
        epochs.event_id = event_id
        epochs.events[:,2] = labels    
    return epochs 

### Data Loading with Band Pass Filtering

In [10]:
# loading relevant files
training_epochs_all = get_mne_epochs_complete(training_files).filter(7,32) # for all training subjects

In [11]:
epochs = training_epochs_all.copy()
data, labels = epochs.get_data(), epochs.events[:,-1]
print('Shape of EEG Data: ', data.shape, '\t Shape of Labels: ', labels.shape) 

Shape of EEG Data:  (800, 12, 3072) 	 Shape of Labels:  (800,)


## Deep Learning with Braindecode 

### It's Training Time with [0.5, 4.5] sec and 2sec window with 1 sec stride (using leave one group out cv)

In [12]:
epochs = training_epochs_all.copy()
epochs = epochs.crop(tmin=0.5, tmax=4.5, include_tmax=False)

In [13]:
from braindecode.datautil import create_from_mne_epochs

# convert epochs to braindecode compatible datastructure 
# 2sec windows with 0.125 sec stride
window_size = 1024 #1024 #1024 #50 # 3072
window_stride = 512 #512 #256 # 50

windows_datasets = create_from_mne_epochs(
            [epochs], # expects list of epochs
            window_size_samples = window_size,
            window_stride_samples = window_stride,
            drop_last_window = False
)

In [15]:
from braindecode.datasets import BaseConcatDataset
import pandas as pd
import numpy as np

def get_windows_datasets_labels(windows_dataset):
    labels = []
    for i in range(len(windows_dataset.datasets)):
        labels.extend(windows_dataset.datasets[i].y)
    return np.array(labels)

# Create your new description DataFrame
description_df = pd.DataFrame(
    data=get_windows_datasets_labels(windows_datasets),
    columns=['labels']
)

# Create a new BaseConcatDataset with the updated description
windows_datasets = BaseConcatDataset(windows_datasets.datasets, description_df)

TypeError: target_transform must be a callable.

In [ ]:
print("Total Windows in a whole Dataset: ", len(windows_datasets.description))

Total Windows in a whole Dataset:  1920


In [ ]:
from braindecode.datautil.preprocess import exponential_moving_standardize
from braindecode.datautil.preprocess import MNEPreproc, NumpyPreproc, preprocess

low_cut_hz = 8.  # low cut frequency for filtering
high_cut_hz = 32.  # high cut frequency for filtering
# Parameters for exponential moving standardization
factor_new = 1e-3
init_block_size = 1000

def custom_exp_moving_std_fn(epochs, factor_new=factor_new, init_block_size=init_block_size):
    data = epochs.get_data()
    for i in range(len(data)):
        epochs._data[i] = exponential_moving_standardize(data[i], 
                        factor_new=factor_new, init_block_size=init_block_size)
    return epochs

preprocessors = [
    # keep only EEG sensors
    MNEPreproc(fn='pick_types', eeg=True, meg=False, stim=False),
    # bandpass filter
    MNEPreproc(fn='filter', l_freq=low_cut_hz, h_freq=high_cut_hz),
    # exponential moving standardization
    MNEPreproc(fn=custom_exp_moving_std_fn, factor_new=factor_new,
        init_block_size=init_block_size)
]

In [ ]:
preprocess(windows_datasets, preprocessors)

In [ ]:
batch_size = 32 #64
n_epochs = 25 #25 #25 #25 #25 #20 #25 use few epochs for quick verification

In [ ]:
# Creating a model
import torch
from braindecode.util import set_random_seeds
from braindecode.models import ShallowFBCSPNet, EEGNetv4

cuda = torch.cuda.is_available()  # check if GPU is available, if True chooses to use it
device = 'cuda' if cuda else 'cpu'
if cuda:
    torch.backends.cudnn.benchmark = True
seed = 20200220  # random seed to make results reproducible
# Set random seed to be able to reproduce results
set_random_seeds(seed=seed, cuda=cuda)

n_classes=2
# Extract number of chans and time steps from dataset
n_chans = windows_datasets[0][0].shape[0]
input_window_samples = windows_datasets[0][0].shape[1]

model = EEGNetv4(
    n_chans,
    n_classes,
    input_window_samples = window_size, #input_window_samples,
    final_conv_length='auto',
)

# Send model to GPU
if cuda:
    model.cuda()

In [ ]:
cv = LeaveOneGroupOut()
# group parameter for leave one group out cross validation in sklearn, each subject is given unique identifier
group_list = []
for subject in np.linspace(1,8,8):
    group_list.extend([subject for _ in range(len(windows_datasets)//8)]) #since total 8 subjects
groups = np.array(group_list)

In [ ]:
# Training time
from skorch.callbacks import LRScheduler
from skorch.helper import predefined_split
from braindecode import EEGClassifier

lr = 1 * 0.02 #0.01 
weight_decay = 0.5 * 0.001

clf =  EEGClassifier(
                    model,
                    criterion=torch.nn.NLLLoss,
                    optimizer=torch.optim.AdamW,
                    #iterator_train = StratifiedShuffleSplit(),#cv.split(epochs, y=labels, groups=groups), 
                    #train_split=predefined_split(train_set),  # using valid_set for validation
                    #train_split = cv.split(epochs, y=labels, groups=groups),
                    train_split = skorch.dataset.CVSplit(cv=cv.split(windows_datasets, 
                          y=windows_datasets.description.labels, groups=groups)),
                    
    
                    #= skorch.dataset.CVSplit(cv=LeaveOneGroupOut())),
                    optimizer__lr=lr,
                    optimizer__weight_decay=weight_decay,
                    batch_size=batch_size,
                    callbacks=[
                        "accuracy", ("lr_scheduler", LRScheduler('CosineAnnealingLR', T_max=n_epochs - 1)),
                    ],
                    device=device,
        
                    )

In [ ]:
def training_function(windows_datasets, n_epochs=25):
    print('\n', '#'*25, 'Cross Subject Training:', '#'*25, '\n')
    dataset = windows_datasets
    clf.fit(dataset, y=dataset.description.labels, epochs=n_epochs);
    best_validation_acc = clf.callbacks_[4][1].best_score_ # a hack to get best validation accuracy
    best_validation_kappa = (2*best_validation_acc)-1
    print("Best Cross Validation Kappa Score: {:.2f}".format(best_validation_kappa))

In [ ]:
training_function(windows_datasets, n_epochs=n_epochs);


 ######################### Cross Subject Training: ######################### 

  epoch    train_accuracy    train_loss    valid_accuracy    valid_loss      lr      dur
-------  ----------------  ------------  ----------------  ------------  ------  -------
      1            0.7464        0.7048            0.6792        0.7166  0.0200  13.4470
      2            0.7417        0.5835            0.6375        1.0621  0.0199  11.6820
      3            0.6690        0.5554            0.5833        1.4334  0.0197  11.2570
      4            0.7833        0.5153            0.7083        0.5871  0.0192  13.7350
      5            0.7976        0.5472            0.6833        0.6751  0.0187  13.5100
      6            0.5304        0.4833            0.5292        1.1845  0.0179  12.6870
      7            0.7173        0.4993            0.6667        0.7118  0.0171  14.0700
      8            0.7554        0.4903            0.7417        0.5515  0.0161  13.0450
      9            0.8321     